<a href="https://colab.research.google.com/github/Jurgo001/TH_TriTueNhanTao/blob/main/Buoi04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
# ----------------------------------------------------------------------
# BƯỚC KHẮC PHỤC LỖI: Cài đặt thư viện ColabTurtle
!pip install ColabTurtle
# ----------------------------------------------------------------------

import numpy as np
import math
import itertools
from ColabTurtle.Turtle import * # Lệnh này sẽ chạy sau khi cài đặt thành công

# --- I. DỮ LIỆU ĐỒ THỊ ---
# Tên các đỉnh
TEN_DINH = "ABCDEF"
SO_DINH = len(TEN_DINH)
INF = float('inf') # Giá trị vô cực

# 1. Ma trận kề (Dùng cho TÔ MÀU)
MA_TRAN_KE = [
    [0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 1, 1], [0, 0, 1, 1, 0, 1], [0, 0, 0, 1, 1, 0]
]

# 2. Ma trận trọng số (Dùng cho TSP)
MA_TRAN_TRONG_SO = [
    [0, 12, 10, 0, 0, 0], [12, 0, 8, 15, 0, 0], [10, 8, 0, 18, 20, 0],
    [0, 15, 18, 0, 10, 25], [0, 0, 20, 10, 0, 30], [0, 0, 0, 25, 30, 0]
]

print("Đã load dữ liệu cho 6 đỉnh:", TEN_DINH)
print("-" * 30)

# Phần còn lại của code (Tô màu, TSP, Vẽ)

def to_mau_do_thi(MA_TRAN):
    # ... (giữ nguyên hàm to_mau_do_thi)
    bac_dinh = [sum(MA_TRAN[i]) for i in range(SO_DINH)]
    chi_so_sap_xep = np.argsort(bac_dinh)[::-1]
    dinh_sap_xep = [TEN_DINH[i] for i in chi_so_sap_xep]

    DS_mau_kha_dung = {dinh: ["Blue", "Red", "Yellow", "Green"] for dinh in TEN_DINH}
    ket_qua_to_mau = {}

    for ten_dinh in dinh_sap_xep:
        if DS_mau_kha_dung[ten_dinh]:
            mau_duoc_chon = DS_mau_kha_dung[ten_dinh][0]
            ket_qua_to_mau[ten_dinh] = mau_duoc_chon

            chi_so_dinh_hien_tai = TEN_DINH.index(ten_dinh)

            for j in range(SO_DINH):
                if MA_TRAN[chi_so_dinh_hien_tai][j] == 1:
                    ten_dinh_ke = TEN_DINH[j]
                    if mau_duoc_chon in DS_mau_kha_dung[ten_dinh_ke]:
                        DS_mau_kha_dung[ten_dinh_ke].remove(mau_duoc_chon)
        else:
            ket_qua_to_mau[ten_dinh] = "Không đủ màu"

    return ket_qua_to_mau

def ve_do_thi_va_to_mau(MA_TRAN, KetQuaMau):
    # ... (giữ nguyên hàm ve_do_thi_va_to_mau)
    initializeTurtle()
    speed(5); bgcolor("white"); pensize(2)

    trung_tam_x, trung_tam_y, ban_kinh = 400, 250, 150
    vi_tri_dinh = {}
    goc_buoc = 360 / SO_DINH

    for i in range(SO_DINH):
        goc_radian = math.radians(i * goc_buoc)
        x = trung_tam_x + math.cos(goc_radian) * ban_kinh
        y = trung_tam_y + math.sin(goc_radian) * ban_kinh
        vi_tri_dinh[TEN_DINH[i]] = (x, y)

    color("gray")
    for i in range(SO_DINH):
        for j in range(i + 1, SO_DINH):
            if MA_TRAN[i][j] == 1:
                toa_do_dau = vi_tri_dinh[TEN_DINH[i]]
                toa_do_cuoi = vi_tri_dinh[TEN_DINH[j]]
                penup(); goto(toa_do_dau); pendown(); goto(toa_do_cuoi)

    for ten_dinh in TEN_DINH:
        x, y = vi_tri_dinh[ten_dinh]
        mau_to = KetQuaMau.get(ten_dinh, "white").lower()

        penup(); goto(x, y); color(mau_to)

        old_size = pensize()
        pensize(30); pendown(); forward(0); pensize(old_size); penup()

        goto(x, y + 8); color("black")
        write(ten_dinh, align="center", font=(20, "Arial", "bold"))

    goto(50, 450)

def tinh_tong_quang_duong(lo_trinh, TRONG_SO):
    # ... (giữ nguyên hàm tinh_tong_quang_duong)
    tong_kc = 0
    for i in range(len(lo_trinh) - 1):
        u = lo_trinh[i]; v = lo_trinh[i+1]
        khoang_cach = TRONG_SO[u][v]
        if khoang_cach == 0: return INF
        tong_kc += khoang_cach
    return tong_kc


# --- THỰC THI ---

# 1. Tô màu và Vẽ kết quả tô màu
KetQuaMau = to_mau_do_thi(MA_TRAN_KE)
print("KẾT QUẢ TÔ MÀU ĐỒ THỊ:")
for t, w in sorted(KetQuaMau.items()):
    print(f"Đỉnh {t} = {w}")

print("-" * 30)
print("Bắt đầu vẽ đồ thị TÔ MÀU...")
ve_do_thi_va_to_mau(MA_TRAN_KE, KetQuaMau)


# 2. Tính toán và Mô phỏng TSP

dinh_con_lai = list(range(1, SO_DINH))
tat_ca_lo_trinh = itertools.permutations(dinh_con_lai)

duong_di_ngan_nhat = []
khoang_cach_ngan_nhat = INF
dem_so_truong_hop = 0

for phuong_an in tat_ca_lo_trinh:
    dem_so_truong_hop += 1
    lo_trinh_hien_tai = [0] + list(phuong_an) + [0]
    do_dai = tinh_tong_quang_duong(lo_trinh_hien_tai, MA_TRAN_TRONG_SO)

    if do_dai < khoang_cach_ngan_nhat:
        khoang_cach_ngan_nhat = do_dai
        duong_di_ngan_nhat = lo_trinh_hien_tai

ten_duong_di = [TEN_DINH[i] for i in duong_di_ngan_nhat]

print("\n--- BÀI TOÁN NGƯỜI DU LỊCH (TSP) ---")
print(f"Đã kiểm tra tổng cộng: {dem_so_truong_hop} trường hợp.")
print(f"-> LỘ TRÌNH TỐI ƯU: {' -> '.join(ten_duong_di)}")
print(f"-> TỔNG QUÃNG ĐƯỜNG: {khoang_cach_ngan_nhat} km")

if khoang_cach_ngan_nhat != INF:
    initializeTurtle()
    speed(13); bgcolor("white"); pensize(2)

    trung_tam_x, trung_tam_y, ban_kinh = 400, 250, 150
    goc_buoc = 360 / SO_DINH
    vi_tri_dinh_tsp = {i: (trung_tam_x + math.cos(math.radians(i * goc_buoc)) * ban_kinh,
                           trung_tam_y + math.sin(math.radians(i * goc_buoc)) * ban_kinh)
                        for i in range(SO_DINH)}

    # Dựng bản đồ có trọng số
    color("lightgray")
    for i in range(SO_DINH):
        for j in range(i + 1, SO_DINH):
            kc = MA_TRAN_TRONG_SO[i][j]
            if kc > 0:
                start = vi_tri_nh_tsp[i]; end = vi_tri_dinh_tsp[j]
                penup(); goto(start); pendown(); goto(end)

                mid_x = (start[0] + end[0]) / 2; mid_y = (start[1] + end[1]) / 2
                penup(); goto(mid_x, mid_y); color("blue")
                write(str(kc), align="center", font=(10, "Arial", "normal")); color("lightgray")

        x, y = vi_tri_dinh_tsp[i]; penup(); goto(x, y + 20); color("black")
        write(TEN_DINH[i], align="center", font=(15, "Arial", "bold"))

    # Mô phỏng đường đi tối ưu
    print("Bắt đầu mô phỏng giao hàng theo lộ trình tối ưu...")
    speed(3); color("red"); pensize(3)

    start_point = vi_tri_dinh_tsp[duong_di_ngan_nhat[0]]
    penup(); goto(start_point); pendown()

    for i in range(1, len(duong_di_ngan_nhat)):
        next_pos = vi_tri_dinh_tsp[duong_di_ngan_nhat[i]]
        goto(next_pos)

    print("Giao hàng hoàn tất!")

penup(); goto(50, 450)

  Preparing metadata (setup.py) ... done
  Created wheel for ColabTurtle: filename=ColabTurtle-2.1.0-py3-none-any.whl size=7643 sha256=39419a8cfa6fce1be8967e938a22bd8c642bf10a8e38495f265417552cf24035
  Stored in directory: /root/.cache/pip/wheels/9f/af/64/ffd85f9858ed7d56b7293dcedbc9d461bf13c8cfc97e352bc8
Successfully built ColabTurtle
Đã load dữ liệu cho 6 đỉnh: ABCDEF
------------------------------
KẾT QUẢ TÔ MÀU ĐỒ THỊ:
Đỉnh A = Red
Đỉnh B = Yellow
Đỉnh C = Blue
Đỉnh D = Red
Đỉnh E = Yellow
Đỉnh F = Blue
------------------------------
Bắt đầu vẽ đồ thị TÔ MÀU...



--- BÀI TOÁN NGƯỜI DU LỊCH (TSP) ---
Đã kiểm tra tổng cộng: 120 trường hợp.
-> LỘ TRÌNH TỐI ƯU: A -> B -> D -> F -> E -> C -> A
-> TỔNG QUÃNG ĐƯỜNG: 112 km


NameError: name 'vi_tri_nh_tsp' is not defined